<a href="https://colab.research.google.com/github/RodBurr/Project-3/blob/main/En_ESModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install datasets
!pip install transformers
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd  # For handling tabular data
from sklearn.feature_extraction.text import CountVectorizer  # For text vectorization

In [ ]:

df_filtered = pd.read_csv("./EN-ES.txt", sep='\t', header = None)[[0,1]].rename(columns = {0:"EN", 1:"ES"})

In [ ]:
# Preprocessing function to clean text using CountVectorizer
def clean_text_sklearn(text):
    # Define a CountVectorizer to convert text to lowercase and remove unwanted characters
    vectorizer = CountVectorizer(lowercase=True, stop_words='english', token_pattern=r'\b\w+\b')

    # Transform the text (vectorization and basic cleaning)
    vectorized = vectorizer.fit_transform([text])

    # Return the cleaned and tokenized text as a string
    cleaned_text = ' '.join(vectorizer.get_feature_names_out())
    return cleaned_text

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_filtered = df_filtered[['EN', 'ES']].dropna()
#df_filtered.columns = ['source', 'target']

# Save for later training
#df_filtered.to_csv("en-es-parallel.csv", index=False)
train_df, test_df = train_test_split(df_filtered, test_size=0.2, random_state=42)

# Save train and test sets
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [ ]:
df_filtered.columns

Index(['EN', 'ES'], dtype='object')

In [ ]:
df_filtered.head(20)

,EN,ES
0,Commission Regulation (EC) No 1788/2004,Reglamento (CE) no 1788/2004 de la Comisión
1,of 15 October 2004,de 15 de octubre de 2004
2,fixing the minimum selling prices for butter f...,por el que se fijan los precios mínimos de ven...
3,"THE COMMISSION OF THE EUROPEAN COMMUNITIES,","LA COMISIÓN DE LAS COMUNIDADES EUROPEAS,"
4,Having regard to the Treaty establishing the E...,Visto el Tratado constitutivo de la Comunidad ...
5,Having regard to Council Regulation (EC) No 12...,Visto el Reglamento (CE) no 1255/1999 del Cons...
6,Whereas:,Considerando lo siguiente:
7,"The intervention agencies are, pursuant to Com...",De conformidad con lo dispuesto en el Reglamen...
8,The amount(s) of the processing securities mus...,El o los importes de las garantías de transfor...
9,The measures provided for in this Regulation a...,Las medidas previstas en el presente Reglament...


In [ ]:
#from datasets import load_dataset

# Load dataset
#dataset = load_dataset("csv", data_files="en-es-parallel.csv")

# Display example
#print(dataset["train"][0])

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, DatasetDict

model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Load datasets from the saved CSV files
train_dataset = load_dataset('csv', data_files={'train': 'train.csv'})['train']
test_dataset = load_dataset('csv', data_files={'test': 'test.csv'})['test']

# Rename columns to 'source' and 'target' for the preprocess_function
train_dataset = train_dataset.rename_columns({'EN': 'input_ids', 'ES': 'labels'})
test_dataset = test_dataset.rename_columns({'EN': 'input_ids', 'ES': 'labels'})
# Combine train and test datasets into a DatasetDict
dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})


def preprocess_function(examples):
    # Filter out empty strings and None values before tokenization
    sources = [src for src in examples["input_ids"] if src and isinstance(src, str)]
    targets = [tgt for tgt in examples["labels"] if tgt and isinstance(tgt, str)]

    # If the batch has valid sources and targets, proceed with tokenization
    if sources and targets:
        inputs = tokenizer(sources, padding="max_length", truncation=True, return_tensors="pt")
        targets = tokenizer(targets, padding="max_length", truncation=True, return_tensors="pt")

        return {"input_ids": inputs["input_ids"], "labels": targets["input_ids"]}
    else:
        # If the batch is empty, return an empty dictionary
        return {}

tokenized_dataset = dataset.map(preprocess_function, batched=True)
# Remove empty examples from the dataset
tokenized_dataset = tokenized_dataset.filter(lambda example: len(example) > 0)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="pt")
)

# Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/322449 [00:00<?, ? examples/s]

Map:   0%|          | 0/80613 [00:00<?, ? examples/s]

Filter:   0%|          | 0/322449 [00:00<?, ? examples/s]

Filter:   0%|          | 0/80613 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: corrianderchris (corrianderchris-michigan-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
